In [8]:
# import libraries
import xarray as xr
from pystac import Item, Collection
import pystac
from datetime import datetime
from shapely.geometry import box, mapping
from xstac import xarray_to_stac
import glob
import json
import shapely
import numpy as np
import geopandas as gpd
import pandas as pd
import time, threading, random
from concurrent.futures import ThreadPoolExecutor
from urllib3.connectionpool import HTTPConnectionPool, HTTPSConnectionPool
from concurrent.futures import ThreadPoolExecutor

In [2]:
%%time
sentinel1_url = 'https://objectstore.eodc.eu:2222/68e13833a1624f43ba2cac01376a18af:ASP_ZARR/S1_out.zarr'
eodc_objectstore_ds = xr.open_zarr(sentinel1_url)
eodc_objectstore_ds

CPU times: user 1.35 s, sys: 237 ms, total: 1.59 s
Wall time: 2.67 s


<xarray.Dataset> Size: 140MB
Dimensions:              (time: 2853, field_id: 1023)
Coordinates:
  * field_id             (field_id) <U14 57kB 'cz_polk_N01-KB' ... 'ua_lviv_3'
    max_lat              (field_id) float64 8kB dask.array<chunksize=(100,), meta=np.ndarray>
    max_lon              (field_id) float64 8kB dask.array<chunksize=(100,), meta=np.ndarray>
    min_lat              (field_id) float64 8kB dask.array<chunksize=(100,), meta=np.ndarray>
    min_lon              (field_id) float64 8kB dask.array<chunksize=(100,), meta=np.ndarray>
    rep_lat              (field_id) float64 8kB dask.array<chunksize=(100,), meta=np.ndarray>
    rep_lon              (field_id) float64 8kB dask.array<chunksize=(100,), meta=np.ndarray>
    row_id               (field_id) object 8kB dask.array<chunksize=(1023,), meta=np.ndarray>
  * time                 (time) datetime64[ns] 23kB 2015-03-01 ... 2022-12-30
Data variables:
    sig0_cr_mean_daily   (time, field_id) float64 23MB dask.array<chunksize=(714, 50), meta=np.ndarray>
    sig0_vh_mean_daily   (time, field_id) float64 23MB dask.array<chunksize=(714, 50), meta=np.ndarray>
    sig0_vv_mean_daily   (time, field_id) float64 23MB dask.array<chunksize=(714, 50), meta=np.ndarray>
    sig40_cr_mean_daily  (time, field_id) float64 23MB dask.array<chunksize=(714, 50), meta=np.ndarray>
    sig40_vh_mean_daily  (time, field_id) float64 23MB dask.array<chunksize=(714, 50), meta=np.ndarray>
    sig40_vv_mean_daily  (time, field_id) float64 23MB dask.array<chunksize=(714, 50), meta=np.ndarray>
Attributes: (12/19)
    country_co:  ua
    crop_damag:  na
    crop_id:     C1111
    crop_type:   common winter wheat
    data_openn:  0
    farm_code:   chmel
    ...          ...
    row_id:      1093
    soil_type:   na
    sowing_dat:  
    tilage:      na
    tilename:    E060N015T3
    yield:       7.93

In [3]:
%%time
pd.Series(eodc_objectstore_ds.sig0_vh_mean_daily.values.flatten()).describe()

CPU times: user 142 ms, sys: 46.5 ms, total: 188 ms
Wall time: 864 ms


count    238075.000000
mean        -19.419959
std           3.285195
min         -31.232473
25%         -21.997556
50%         -19.219452
75%         -16.864764
max          -8.850440
dtype: float64

In [2]:
prr_ds = xr.open_dataset('remote_kerchunk_zip.json', engine='kerchunk', chunks={})
prr_ds

<xarray.Dataset> Size: 140MB
Dimensions:              (time: 2853, field_id: 1023, row_id: 1023)
Coordinates:
  * field_id             (field_id) <U14 57kB 'cz_polk_N01-KB' ... 'ua_lviv_3'
    max_lat              (field_id) float64 8kB dask.array<chunksize=(100,), meta=np.ndarray>
    max_lon              (field_id) float64 8kB dask.array<chunksize=(100,), meta=np.ndarray>
    min_lat              (field_id) float64 8kB dask.array<chunksize=(100,), meta=np.ndarray>
    min_lon              (field_id) float64 8kB dask.array<chunksize=(100,), meta=np.ndarray>
    rep_lat              (field_id) float64 8kB dask.array<chunksize=(100,), meta=np.ndarray>
    rep_lon              (field_id) float64 8kB dask.array<chunksize=(100,), meta=np.ndarray>
  * row_id               (row_id) <U34 139kB '20200,20276,20325,20381' ... '1...
  * time                 (time) datetime64[ns] 23kB 2015-03-01 ... 2022-12-30
Data variables:
    sig0_cr_mean_daily   (time, field_id) float64 23MB dask.array<chunksize=(714, 50), meta=np.ndarray>
    sig0_vh_mean_daily   (time, field_id) float64 23MB dask.array<chunksize=(714, 50), meta=np.ndarray>
    sig0_vv_mean_daily   (time, field_id) float64 23MB dask.array<chunksize=(714, 50), meta=np.ndarray>
    sig40_cr_mean_daily  (time, field_id) float64 23MB dask.array<chunksize=(714, 50), meta=np.ndarray>
    sig40_vh_mean_daily  (time, field_id) float64 23MB dask.array<chunksize=(714, 50), meta=np.ndarray>
    sig40_vv_mean_daily  (time, field_id) float64 23MB dask.array<chunksize=(714, 50), meta=np.ndarray>
Attributes: (12/19)
    country_co:  ua
    crop_damag:  na
    crop_id:     C1111
    crop_type:   common winter wheat
    data_openn:  0
    farm_code:   chmel
    ...          ...
    row_id:      1093
    soil_type:   na
    sowing_dat:  
    tilage:      na
    tilename:    E060N015T3
    yield:       7.93

In [3]:
%%time
pd.Series(prr_ds.sig0_vh_mean_daily.values.flatten()).describe()

CPU times: user 190 ms, sys: 41.2 ms, total: 231 ms
Wall time: 798 ms


count    238075.000000
mean        -19.419959
std           3.285195
min         -31.232473
25%         -21.997556
50%         -19.219452
75%         -16.864764
max          -8.850440
dtype: float64

In [6]:
# --- workload: eodc ---
def _job():
    sentinel1_url = 'https://objectstore.eodc.eu:2222/68e13833a1624f43ba2cac01376a18af:ASP_ZARR/S1_out.zarr'
    eodc_objectstore_ds = xr.open_zarr(sentinel1_url)
    # Touch the data to force IO; .values will pull the referenced bytes
    out = pd.Series(eodc_objectstore_ds.sig0_vh_mean_daily.values.flatten()).describe()
    eodc_objectstore_ds.close()
    return out


In [ ]:
# --- workload: your kerchunk/xarray open + compute ---
def _job():
    ds = xr.open_dataset('remote_kerchunk_zip.json', engine='kerchunk', chunks={})
    # Touch the data to force IO; .values will pull the referenced bytes
    out = pd.Series(ds.sig0_vh_mean_daily.values.flatten()).describe()
    ds.close()
    return out


In [ ]:
%%time
CONCURRENCY = 5
JITTER_MAX_S = 0.125          # up to 125ms random delay before each request

REQS = 0
FAILS = 0
_LOCK = threading.Lock()

def _hit(ok=True):
    global REQS, FAILS
    with _LOCK:
        if ok: REQS += 1
        else:  FAILS += 1

# --- patch aiohttp (used by fsspec/http + kerchunk async paths) ---
try:
    import asyncio, aiohttp
    _orig_aio = aiohttp.client.ClientSession._request
    async def _wrap_aio(self, method, url, *a, **k):
        try:
            # gentle staggering
            if JITTER_MAX_S > 0:
                await asyncio.sleep(random.random() * JITTER_MAX_S)
            _hit(True)
            return await _orig_aio(self, method, url, *a, **k)
        except Exception:
            _hit(False)
            raise
    aiohttp.client.ClientSession._request = _wrap_aio
except Exception:
    pass

# --- patch requests / urllib3 (many libs sync path) ---
try:
    import requests
    _orig_req = requests.Session.request
    def _wrap_request(self, method, url, *a, **k):
        try:
            if JITTER_MAX_S > 0:
                time.sleep(random.random() * JITTER_MAX_S)
            _hit(True)
            return _orig_req(self, method, url, *a, **k)
        except Exception:
            _hit(False)
            raise
    requests.Session.request = _wrap_request
except Exception:
    pass

try:
    from urllib3.connectionpool import HTTPConnectionPool, HTTPSConnectionPool
    _http_orig  = HTTPConnectionPool.urlopen
    _https_orig = HTTPSConnectionPool.urlopen
    def _wrap_urlopen(orig):
        def _u(self, method, url, *a, **k):
            try:
                if JITTER_MAX_S > 0:
                    time.sleep(random.random() * JITTER_MAX_S)
                _hit(True)
                return orig(self, method, url, *a, **k)
            except Exception:
                _hit(False)
                raise
        return _u
    HTTPConnectionPool.urlopen  = _wrap_urlopen(_http_orig)
    HTTPSConnectionPool.urlopen = _wrap_urlopen(_https_orig)
except Exception:
    pass


start = time.perf_counter()
with ThreadPoolExecutor(max_workers=CONCURRENCY) as ex:
    results = list(ex.map(lambda _: _job(), range(CONCURRENCY)))
elapsed = time.perf_counter() - start

# --- unpatch (optional good hygiene) ---
try: aiohttp.client.ClientSession._request = _orig_aio
except: pass
try: requests.Session.request = _orig_req
except: pass
try:
    HTTPConnectionPool.urlopen  = _http_orig
    HTTPSConnectionPool.urlopen = _https_orig
except: pass

print(f"Parallel runs: {len(results)}")
print(f"Total wall time (s): {elapsed:.3f}")
print(f"HTTP requests counted: {REQS}  | failures: {FAILS}")
print(f"Throughput (req/s): {REQS/elapsed:.2f}" if REQS else "Throughput (req/s): n/a")



Parallel runs: 5
Total wall time (s): 4.185
HTTP requests counted: 565  | failures: 0
Throughput (req/s): 135.00
CPU times: user 773 ms, sys: 238 ms, total: 1.01 s
Wall time: 4.19 s
